In [1]:
pip install loguru

Note: you may need to restart the kernel to use updated packages.


In [1]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
ua = UserAgent
import requests
import pandas as pd
import lxml
import numpy as np
import threading
import os
import time
import math
from loguru import logger

In [2]:
logger.add('file_{time}.log', format = '{name} {message}', level = 'DEBUG', rotation = '5 MB')
ua = UserAgent(browsers = ['edge', 'chrome', 'opera', 'safari'])
headers = {'User-Agent': ua.random}
urlsBooks = []
allBooks = []
page_num_all = []
filteredBooks = pd.DataFrame(columns = ['id', 'Name', 'Author', 'Publisher', 'Rating', 'OldPrice', 'NewPrice', 'path',
                                        'genre', 'yearOfIssue', 'numberOfPage', 'ageLimit'])

## Получение кол-ва страниц на сайте
link = 'https://www.chitai-gorod.ru/catalog/books-18030'
page = requests.get(link, headers = headers)
soup = BeautifulSoup(page.text, 'lxml')
pagination = soup.find('div', class_ = 'pagination')
page_num_all = pagination.find_all('span', class_ = 'pagination__text')
count_of_page = page_num_all[-1]
count_of_page = count_of_page.get_text(strip = True)
count_of_page = int(count_of_page) + 1
    
#Кол-во страниц на один поток
range_links = math.ceil(count_of_page / os.cpu_count())

In [ ]:
#Получение ссылок на книги со страницы
@logger.catch
def links_get(n):
    n = n * range_links
    k = n + range_links
    if n == 0:
        n = 1
        
    for x in range(n, k):
        link_page = 'https://www.chitai-gorod.ru/catalog/books-18030?page=' + str(x)
        page = requests.get(link_page, headers = headers)
        
        if page.status_code != 200:
            page = requests.get(link_page, headers = headers)
            time.sleep(10)
            
        soup = BeautifulSoup(page.text, 'lxml')
        product_list = soup.find('div', class_ = 'products-list')
        
        if product_list is None:
            page = requests.get(link_page, headers = headers)
            time.sleep(10)
            soup = BeautifulSoup(page.text, 'lxml')
            product_list = soup.find('div', class_ = 'products-list')
            
        allBooks = product_list.find_all('article')
        
        for i in allBooks:
            links = 'https://www.chitai-gorod.ru' + i.find(class_ = 'product-card__picture product-card__row')['href']
            urlsBooks.append(links)

def run_threads():
    threads = []
    n = 0
    
    for x in range(os.cpu_count()):   
        t = threading.Thread(target = links_get, args = (n,))
        n += 1
        t.start()
        threads.append(t)

    for t in threads:
        t.join()

if __name__ == '__main__':
        run_threads()

In [ ]:
range_book_url = math.ceil(len(urlsBooks) / os.cpu_count())

#Получение информации о книге с ее страницы
@logger.catch
def book_info(n):
    print(f'Поток №{n}')
    n = n * range_book_url
    k = n + range_book_url
    
    urlsBooks_info = urlsBooks[n:k:]
    
    for x in urlsBooks_info:
        book = []
        book_url = x
        page = requests.get(book_url, headers = headers)
        soup = BeautifulSoup(page.text, 'lxml')
        pathAll = []
        genreAll = []
        id_tag = soup.find('span', class_='product-detail-characteristics__item-value')
        name_tag = soup.find('h1', class_='app-title app-title--mounted product-detail-title__header app-title--header-4')
        author_tag = soup.find('a', class_='product-detail-title__author')
        izd_tag = soup.find('a', class_='product-detail-characteristics__item-value product-detail-characteristics__item-value--link')
        rating_tag = soup.find('span', class_='review-range__count')
        oldPrice_tag = soup.find('div', class_='product-detail-offer-header__old-price')
        newPrice_tag = soup.find('span', class_='product-detail-offer-header__price-currency product-detail-offer-header__price-currency--sale')
        year_tag = soup.find('span', itemprop = 'datePublished')
        numberOfPages_tag = soup.find('span', itemprop = 'numberOfPages')
        typicalAgeRange_tag = soup.find('span', itemprop = 'typicalAgeRange')
        genre_tag = soup.findAll('ul', class_='app-breadcrumbs product-detail-header__breadcrumbs')
        
        for i in genre_tag:
            temp = i.get_text()
            pathAll.append(temp)
            
        #Форматирвание строки с жанром 
        genreAll = np.array_split(pathAll, len(pathAll))
        genreAll_str = str(genreAll[0])
        genreAll_str = genreAll_str.replace('\\n','')
        genreAll_str = genreAll_str.replace('             ','')
        genreAll_str = genreAll_str.replace("'","")
        genreAll_str = genreAll_str.replace('      ','')
        genreAll_str = genreAll_str.replace('[','')
        genreAll_str = genreAll_str.replace(']','')
        genreAll = genreAll_str.split('   ')
        
        
        id = str(id_tag.contents[0])
        id = id.replace('\n', '')
        id = id.strip()
        
        book.append(id)

        name = str(name_tag.contents[0])
        name = name.replace('\n', '')
        name = name.strip()

        book.append(name)

        if author_tag is not None:
            author = str(author_tag.contents[0])
            author = author.replace('\n', '')
            author = author.strip()
        else:
            author = 'Без автора'
            
        book.append(author)

        if izd_tag is not None:
            izd = str(izd_tag.contents[0])
            izd = izd.replace('\n', '')
            izd = izd.strip()
        else:
            izd = 'Без издательства'

        book.append(izd)

        if rating_tag is not None:
            rating = str(rating_tag.contents[0])
            rating = rating.replace('\n', '')
            rating = rating.strip()
        else:
            rating = 'Без рейтинга'
            
        book.append(rating)

        if oldPrice_tag is not None:
            oldPrice = str(oldPrice_tag.contents[0])
            oldPrice = oldPrice.replace('\n', '')
            oldPrice = oldPrice.strip()
        else:
            oldPrice = 'Без цены'
        
        book.append(oldPrice)

        if newPrice_tag is not None:
            newPrice = str(newPrice_tag.contents[0])
            newPrice = newPrice.replace('\n', '')
            newPrice = newPrice.strip()
        else:
            newPrice = 'Без цены'

        book.append(newPrice)

        path = pathAll[0]
        path = path.replace('\n', '')
        path = path.replace('   ', '')
        path = path.strip()

        book.append(path)
        
        genre = genreAll[-1]

        book.append(genre)

        if year_tag is not None:
            year = str(year_tag.contents[0])
            year = year.replace('\n', '')
            year = year.strip()
        else:
            year = 'Без года'

        book.append(year)
        
        if numberOfPages_tag is not None:
            numberOfPages = str(numberOfPages_tag.contents[0])
            numberOfPages = numberOfPages.replace('\n', '')
            numberOfPages = numberOfPages.strip()
        else:
            numberOfPages = 'Без кол-ва стр'
        
        book.append(numberOfPages)

        if typicalAgeRange_tag is not None:
            typicalAgeRange = str(typicalAgeRange_tag.contents[0])
            typicalAgeRange = typicalAgeRange.replace('\n', '')
            typicalAgeRange = typicalAgeRange.strip()
        else:
            typicalAgeRange = 'Без возрастных ограничений'

        book.append(typicalAgeRange)

        filteredBooks.loc[len(filteredBooks)] = book

def run_threads_book():    
    threads = []
    n = 0 
    
    for x in range(os.cpu_count()):
        t = threading.Thread(target = book_info, args = (n,))
        t.start()
        threads.append(t)
        n += 1
        
    for t in threads:
        t.join()

if __name__ == '__main__':
    
    run_threads_book()

filteredBooks.to_csv('parserBook.csv')